__#7. Магические методы setattr, getattribute, getattr и delattr__

свойства и методы - атрибуты класса

атрибуты класса являются общими для всех его экземпляров, так как они содержат ссылку на них

мы не можем к атрибутам класса обращаться по именам, нужно или через self или через название класса (не рекомендуется)

def set_bound(self, left):
        self.MIN_COORD = left
не изменяет атрибут класса, а создает новое локальное свойство для экземпляра. Правильнее использовать @classmethod и ссылку на класс (cls)

setattr(self, key, value) - автоматически вызывается при изменении свойства key класса. Можем запретить создавать какой-либо локальный атрибут в экземплярах класса.

getattribute(self, item) - автоматически вызывается при получении свойства класса с именем item. Используется например для запрета обращения к какому-либу атрибуту

getattr(self, item) - автоматически вызывается при получении несуществующего свойства item класса. Ну эт используется если там чет нужно продумать при обращении к несуществующему элементу класса. Чтоб допустим возвращалось False, а не исключение дропалось.

delattr(self, item) - автоматически вызывается при удалении свйоства item (не важно существует оно или нет). Контролировать удаление тех или иных атрибутов

In [7]:
class Point():
    MAX_COORD = 100
    MIN_COORD = 0

    def __init__(self, x, y):
        self.x = x
        self.y = y 
    
    def set_coord(self, x, y):
        self.x = x 
        self.y = y

    @classmethod
    def set_bound(cls, left):
        cls.MIN_COORD = left

    def __getattribute__(self, item):
        if item == "x":
            raise ValueError("доступ запрещен")
        else: 
            return object.__getattribute__(self, item)

    def __setattr__(self, name, value):
        if name == 'z': 
            raise AttributeError("недопустимое имя атрибута")
        else:
            object.__setattr__(self, name, value)

    def __getattr__(self, name):
        return False

    def __delattr__(self, name):
        object.__delattr__(self, name)

pt1 = Point(1, 2)
pt1.set_bound(100)
print(pt1.__dict__)
print(Point.__dict__)

a = pt1.y
print(a)

print(pt1.yy)

del Point

{'x': 1, 'y': 2}
{'__module__': '__main__', '__firstlineno__': 1, 'MAX_COORD': 100, 'MIN_COORD': 100, '__init__': <function Point.__init__ at 0x000001EDCC02E3E0>, 'set_coord': <function Point.set_coord at 0x000001EDCC02E7A0>, 'set_bound': <classmethod(<function Point.set_bound at 0x000001EDCC02EE80>)>, '__getattribute__': <function Point.__getattribute__ at 0x000001EDCC02E520>, '__setattr__': <function Point.__setattr__ at 0x000001EDCC02EFC0>, '__getattr__': <function Point.__getattr__ at 0x000001EDCC02F060>, '__delattr__': <function Point.__delattr__ at 0x000001EDCC02F100>, '__static_attributes__': ('x', 'y'), '__dict__': <attribute '__dict__' of 'Point' objects>, '__weakref__': <attribute '__weakref__' of 'Point' objects>, '__doc__': None}
2
False


__#8. Паттерн "Моносостояние"__

Идея заключается в том, что изменение атрибутов класса в каком-либо объекте, изменится у всех объектов

Очень редко используется на практике

In [8]:
class ThreadData:
    __shared_attrs= {
        'name': 'thread_1',
        'data': {},
        'id' : 1
    }

    def __init__(self):
        self.__dict__ = self.__shared_attrs

th1 = ThreadData()
th2 = ThreadData()

th1.id = 3 
print(th2.id)

th1.attr_new = 'new_attr'

del ThreadData

3


__#9. Свойства property. Декоратор @property__

Короче переопределяет функции в зависимости от функционала, например чтоб при вызове одной функции одним способом можно получить значение (геттер), а также можно и записать значения (сеттер) и тому подобное 

У свойства property приоритет выше, чем при обращении к приватному атрибуту экземпляра класса.

In [9]:
class Person:
    def __init__(self, name, old):
        self.__name = name
        self.__old = old 

    def get_old(self):
        return self.__old 
    
    def set_old(self, old):
        self.__old = old 

    old = property(get_old, set_old)

p = Person('Сергей', 20)

a = p.old
print(a)

p.old = 35 
print(p.old)


del Person 

20
35


In [10]:
# как чаще всего используют все на практике:
class Person:
    def __init__(self, name, old):
        self.__name = name
        self.__old = old 

    @property
    def old(self):
        return self.__old 
    
    @old.setter
    def old(self, old):
        self.__old = old

    @old.deleter 
    def old(self):
        del self.__old 


p = Person('Сергей', 20)

a = p.old
print(a)

p.old = 35 
print(p.old)


del Person 

20
35


__#10. Пример использования объектов property__

if len(s.strip(letters)) != 0:  - если fio содержит разрешенные символы из letters, то стрип их все удалит и длина будет равна нулю

In [11]:
from string import ascii_letters

class Person: 
    S_RUS = 'йцукенгшщзхъфывапролджэячсмитьбю'
    S_RUS_UPPER = S_RUS.upper()
    
    def __init__(self, fio, old, ps, weight): 
        self.verify_fio(fio)
        self.verify_old(old)
        self.verify_ps(ps)
        self.verify_weight(weight)

        self.__fio = fio.split()
        self.__old = old 
        self.__passport = ps 
        self.__weight = weight 

    @classmethod 
    def verify_fio(cls, fio):
        if type(fio) != str: 
            raise TypeError("ФИО должно быть строкой")
        
        f = fio.split() 
        if len(f) != 3: 
            raise TypeError("Неверный формат ФИО")
        
        letters = ascii_letters + cls.S_RUS + cls.S_RUS_UPPER
        for s in f: 
            if len(s) < 1:
                raise TypeError('В ФИО lолжен быть хотя бы один символ')
            if len(s.strip(letters)) != 0: 
                raise TypeError('В ФИО должны быть допустимые символы')
            
    @classmethod
    def verify_old(cls, old):
        if type(old) != int or old < 14 or old > 120:
            raise TypeError('Возраст должен быть целым числом в диапозоне [14; 120]')
        
    @classmethod
    def verify_weight(cls, weight):
        if type(weight) != float or weight < 20:
            raise TypeError('Вес должен быть вещественным числом больше 20')
        
    @classmethod 
    def verify_ps(cls, ps):
        if type(ps) != str: 
            raise TypeError('Паспорт должен быть строкой')
        
        s = ps.split()
        if len(s) != 2 or len(s[0]) != 4 or len(s[1]) != 6: 
            raise TypeError('Неверный формат паспорта')

        for p in s: 
            if not p.isdigit():
                raise TypeError('Серия и номер паспорта должны быть числами')

    @property
    def fio(self):
        return self.__fio 
    
    @property
    def old(self):
        return self.__old 
    
    @old.setter 
    def old(self, old):
        self.verify_old(old)
        self.__old = old 

    @property
    def weight(self):
        return self.__weight 
    
    @weight.setter 
    def weight(self, weight):
        self.verify_weight(weight)
        self.__weight = weight 

    @property
    def passport(self):
        return self.__passport 
    
    @passport.setter 
    def passport(self, ps):
        self.verify_ps(ps)
        self.__passport = ps 
    

p = Person('Балакирев Сергей Михайлович', 30, '1234 567890', 80.0)
p.old = 100 
p.passport = '4567 123456'
p.weight = 70.0
print(p.__dict__)

del Person

{'_Person__fio': ['Балакирев', 'Сергей', 'Михайлович'], '_Person__old': 100, '_Person__passport': '4567 123456', '_Person__weight': 70.0}


__#11. Дескрипторы (data descriptor и non-data descriptor)__

Non-data descriptor - класс у которого есть только геттер (имеет тот же приоритет, что и атрибуты класса)

Data descriptor - класс, у которого есть геттер, сеттер и делитер

крч очень помогают сократить код чтоб не писать по 100 проперти и для каждого пука сеттер и геттер

In [12]:
class Integer:
    @classmethod
    def verify_coord(cls, coord): 
        if type(coord) != int: 
            raise TypeError('Координата должна быть числом')
        
    def __set_name__(self, owner, name):
        self.name = "_" + name
 
    def __get__(self, instance, owner):
        return getattr(instance, self.name)
      
    def __set__(self, instance, value):
        self.verify_coord(value)
        print(f"__set__: {self.name} = {value}")
        setattr(instance, self.name, value)

class Point3D:
    x = Integer()
    y = Integer()
    z = Integer()
    
    def __init__(self, x, y, z):
        self.x = x 
        self.y = y 
        self.z = z

p = Point3D(1, 2, 3)
print(p.__dict__)

del Point3D

__set__: _x = 1
__set__: _y = 2
__set__: _z = 3
{'_x': 1, '_y': 2, '_z': 3}
